#  problem formulation

* **The problem**  

Is model is able to predict to Whether   specific reddit post is fake news or not, by looking at its title.
* we have only one feature( we can consider only  one input (text)as **input** to dataset and **output** is prediction of the probability of a specific reddit post is fake news or no (0-1, float) from column called label(only one output) .the data mining function that we need in this problems is classification  predictions.
* **The challenges**  on business understanding,data selection,data preparation because  The data is raw (contains various forms of words) so it needs some of text preprocessing techniques to be applied.
* Challenges according to real life  are difficult to differentiate fake news or not.

* **The impact of the problem** on real life help us to solve social problem by knowing fake news or not  as this problem causes many problems especially in social network . so we can get to **ideal solution**  if  we could get  to fake news or not, by looking at its title.
*  **Note**: according to the problem ,I got to the ideal solution Random forest with tunning hyper paramter  with random search(0.88030 private score on kaggle but late submission ) and some of preprocessing that I will talk about it later.

 with some of preprocessing on text.
. So we apply life cycle of data mining to be able to predict Whether specific reddit post is fake news or not, by looking at its title.

Data collection.

Data selection.

Data preparation.

Modeling.

Evaluation.

Development.




# The experimental protocol that I applied it .

* 1- Using  hold out method that Indicates to spliting training data to train and test split to be able to measure performance.
 -  With a training set, a validation set, and a testing set
- Training set is used for training
- Validation used for error estimation
 - Based on the estimated error, adjust hyperparameters
-Testing set used for final testing (like the leaderboard)
* 2-Using grid search ,randomsearch, for tunning hyper paramters with validation set.
* 3- Using Tfi-df vectorizer to Convert a collection of raw documents to a matrix of TF-IDF features  so the modele is able to predict( word-level vectorizer.).
* 4-Using count vectorizer to Convert a collection of raw documents to numeric features  so the modele is able to predict(character-level vectorizer ).
# Preprocessing steps that I need:


* I used  pipline  to apply these steps (vectorizer or transformer,model training ).

* **Preprocessing steps** .

* - remove any html tags (< /br> often found)
 * - Keep only ASCII + European Chars and whitespace, no digits
  * - remove single letter chars
 * - convert all whitespaces (tabs etc.) to single wspace
  * if not for embedding (but e.g. tdf-idf):
  * - all lowercase
  * - remove stopwords, punctuation and stemm,**Lemmatization.**
 * **NOTE**:I found that Lemmatization  is very useful for  our dataset.


#Data preparation

#Loading and Visualizing the Data
We first load the packages as usual.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path


#plan for try 1 logistic
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for transforming text to numerical using **TfidfVectorizer,**training model(**logistic regression  classifier** with  tunning hyperparamter with **random search with validation set**).


* In this try :I used different cleaning  steps on text.
* steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm



#Observation :
* model with tunning  hyper paramter  with **random search.**
* I expect little best  because **Stemming** is the process of reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words ,
* it is not useful for the data set that i have so i will try **Lemmatization**  to increase performance in the second try and actually  the  private score  of using stemmer  with **logistic** on kaggle(0.83928 ).




In [ ]:
# Read  data

data = pd.read_csv("/content/xy_train.csv", sep=",", na_values=[""])
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
data .info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
data.head(5)# print first 5 rows

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [ ]:
data=data.drop(labels='id', axis=1)# drop idt column (it  may not be useful)

## Cleaning and pre processing

Having consistent and clean data is fundamental for good modeling results. No matter how sophisticated your model the basic principle is: trash in trash out. When dealing with NLP the cleaning and pre processing can differ depending on which model you intend to use. We will use frequency based representation methods for our text. Thus, we usually want to have a pretty thorough manipulation of the input data:

In [ ]:
nltk.download('punkt')# download punkt from ntlk
nltk.download('stopwords')## download stopwords from ntlk

stemmer = SnowballStemmer("english")# define SnowballStemmer algorithm
stop_words = set(stopwords.words("english"))# define stop_words in english


def clean_text(text, for_embedding=False):# apply clean function.
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    #applay regular expression(re) is mostly used library for text cleaning
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)
#	Replaces one or many matches with astring using sub RegEx Functions
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
#convert text to lowercase
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This transformation has a few benefits. Removing characters and words that don't hold much meaning reduces the size of our data. Moreover, it can improve prediction performance when modeling by lowering the noise in the data. This is because e.g. stop words like prepositions or punctuation won't allow our model to extract additional information / meaning (at least when using simple models). By stemming and lower casing words we make sure that similar words are treated identically. Thus, we can improve model performance again by increasing the number of relevant data points.  
Let's apply this to our data:

In [ ]:
%%time
# apply clean function with  lambda  function
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 32.4 s, sys: 118 ms, total: 32.6 s
Wall time: 40.1 s


In [ ]:
print(list(data["label"].values))#print values of label

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [ ]:
data.drop(data.loc[data["label"]==2].index,inplace=True)# drop value of 2 in label as it is found least number of times.

In [ ]:

data_clean = data.copy()# make a copy


These steps conclude the cleaning and pre processing. In result, we get this:

In [ ]:
data_clean.head(3)



,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group friend began volunt homeless shelter nei...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minist theresa may nerv attack f...
2,"In 1961, Goodyear released a kit that allows P...",0,goodyear releas kit allow ps brought heel http...


## Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data_clean["text_clean"]).split()).value_counts()
word_freq[1:40]

one         3285
like        3128
new         2998
look        2847
color       2737
man         2729
get         2602
trump       2578
say         2347
peopl       2316
use         2307
first       2248
make        2227
old         2226
time        2027
poster      2000
found       1999
day         1935
war         1858
post        1648
world       1570
work        1531
show        1513
us          1506
american    1504
take        1491
life        1482
psbattl     1470
help        1442
go          1420
state       1409
back        1369
two         1364
school      1345
see         1329
photo       1324
made        1314
right       1311
save        1308
dtype: int64

In [ ]:
type(word_freq)#check type

pandas.core.series.Series

In [ ]:
print(data_clean["text_clean"].duplicated().sum())

453


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,angriff,1
1,delusion,1
2,wane,1
3,undament,1
4,miku,1
5,hatsun,1
6,nfler,1
7,hicock,1
8,mccall,1
9,wahr,1


In [ ]:
# Distribution of label
data_clean["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

In [ ]:
data_clean.shape# check shape
data_clean.head(3)# print first 3 rows

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group friend began volunt homeless shelter nei...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minist theresa may nerv attack f...
2,"In 1961, Goodyear released a kit that allows P...",0,goodyear releas kit allow ps brought heel http...


In [ ]:
data_clean.info()# check types and information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59768 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        59768 non-null  object
 1   label       59768 non-null  int64 
 2   text_clean  59768 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:



y = data_clean['label'] # target

X = data_clean["text_clean"]#feature
print('original shape', X.shape, y.shape)




original shape (59768,) (59768,)


In [ ]:
!pip install scikit_optimize  #install skopt

## Feature creation with TF-IDF
* Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation using  apply TF-IDF using scikit-learn.
* The Pipeline object encapsulates several processing steps into one(we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search or random search functionality in sci-kit learn.

Randomsearch with validation set

In [ ]:
# feature creation and modelling in a single function
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022) # split data
    # Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit

pds = PredefinedSplit(test_fold = split_index)
# apply tranformer ,model using pipline.
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("LogisticRegression", LogisticRegression(verbose=1, solver='liblinear',random_state=42,  penalty='l2',max_iter=1000))])


# define parameter space to test 

params = {
   "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 15),
    'LogisticRegression__C': [0.1, 1],  
    
    
    
}


# here we still use X_train; but the random search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set

pipe_clf = RandomizedSearchCV(
    pipe, params, n_jobs=-1, scoring='roc_auc',cv=pds)
pipe_clf.fit(X, y)


[LibLinear]

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1,  0])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('LogisticRegression',
                                              LogisticRegression(max_iter=1000,
                                                                 random_state=42,
                                                                 solver='liblinear',
                                                                 verbose=1))]),
                   n_jobs=-1,
                   param_distributions={'LogisticRegression__C': [0.1, 1],
                                        'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                                        'tfidf__ngram_range': [(1, 2), (1, 3)]},
                   scoring='roc_auc')

In [ ]:
print(pipe_clf.best_params_)

{'tfidf__ngram_range': (1, 3), 'tfidf__min_df': 6, 'tfidf__max_df': 0.3, 'LogisticRegression__C': 1}


In [ ]:
print('best score {}'.format(pipe_clf.best_score_))

best score 0.8682520072989361


##load  test data and do the same pre processing.

In [ ]:
data__test = pd.read_csv("/content/x_test.csv")# read data
data__test

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
datats_clean = data__test.copy()# make acopy

In [ ]:
data__test.drop('id', axis=1,inplace=True)# drop idt column (it  may not be useful)

In [ ]:
data__test.head(2)# print first 2 columns of dataframe

,text
0,stargazer
1,yeah


In [ ]:
%%time
# Aplly function of clean using   lambda function
data__test["text_clean"] = data__test.loc[data__test["text"].str.len() > 0, "text"]
data__test["text_clean"] = data__test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 17.3 s, sys: 53.8 ms, total: 17.3 s
Wall time: 19.5 s


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,angriff,1
1,delusion,1
2,wane,1
3,undament,1
4,miku,1
5,hatsun,1
6,nfler,1
7,hicock,1
8,mccall,1
9,wahr,1


In [ ]:
print(data__test["text_clean"])# print text after cleaning

0                                                stargaz
1                                                   yeah
2         pd phoenix car thief get instruct youtub video
3                   trump accus iran one problem credibl
4                                       believ hezbollah
                              ...                       
59146                       bicycl taxi driver new delhi
59147               trump blow gop formula win hous race
59148    napoleon return exil island elba march colouris
59149                      deep alway want ballet dancer
59150         toddler miracul surviv stori fall land car
Name: text_clean, Length: 59151, dtype: object


In [ ]:
test_pred=pipe_clf.predict_proba(data__test["text_clean"])# prediction on all dataset
test_pred

array([[0.57545282, 0.42454718],
       [0.85449478, 0.14550522],
       [0.43181457, 0.56818543],
       ...,
       [0.94555847, 0.05444153],
       [0.72874593, 0.27125407],
       [0.15393726, 0.84606274]])

Save result in csv file

In [ ]:
submission = pd.DataFrame()

submission['id'] = datats_clean["id"]

submission['label'] = pipe_clf.predict_proba(data__test["text_clean"])[:,1]

submission.to_csv('try77.csv', index=False)

#plan for try 2 logistic
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for transforming text to numerical using **TfidfVectorizer**,training model(**logistic regression**  classifier with  **tunning hyperparamter** **with grid** **with validation set**).

* In this try 
* I used different cleaning ( Lemmatization)steps on text.
steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation ,**Lemmatization **



#Observation :
* model with tunning  hyper paramter  with gridsearch .
* I expect   that try is better than one  because **Lemmatization** doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . 
 and actually  the  private score  of **using Lemmatization** with logistic on kaggle(0.85928 ).
*  In the third try , I will use XGB to increase the performance of the model.




In [ ]:
# Read  data

data = pd.read_csv("/content/xy_train.csv", sep=",", na_values=[""])
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
data .info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
data.head(5)# print first 5 rows

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [ ]:
data=data.drop(labels='id', axis=1)# drop idt column (it  may not be useful)

##Cleaning and pre processing

Having consistent and clean data is fundamental for good modeling results. No matter how sophisticated your model the basic principle is: trash in trash out. When dealing with NLP the cleaning and pre processing can differ depending on which model you intend to use. We will use frequency based representation methods for our text. Thus, we usually want to have a pretty thorough manipulation of the input data:

In [ ]:
 
import nltk 
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:

nltk.download('punkt')# download punkt from ntlk
nltk.download('stopwords')## download stopwords from ntlk

stemmer = SnowballStemmer("english")# define SnowballStemmer algorithm
stop_words = set(stopwords.words("english"))# define stop_words in english

def _lemmatize( text):# apply lemmatize function.

  toekn_lemma=[]
  for w in text:
      toekn_lemma.append(wordnet_lemmatizer.lemmatize(w))
  return ' '.join(toekn_lemma)
# apply clean function
		
def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and lemmatize
    """
    #applay regular expression(re) is mostly used library for text cleaning
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)
#	Replaces one or many matches with astring using sub RegEx Functions
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
#convert text to lowercase
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    
    text_clean=_lemmatize(words_tokens_lower)
    return text_clean
    
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The `clean_text` function takes a string input and applies a bunch of manipulations to it (described in the code). Check out this example:

This transformation has a few benefits. Removing characters and words that don't hold much meaning reduces the size of our data. Moreover, it can improve prediction performance when modeling by lowering the noise in the data. This is because e.g. stop words like prepositions or punctuation won't allow our model to extract additional information / meaning (at least when using simple models). By stemming and lower casing words we make sure that similar words are treated identically. Thus, we can improve model performance again by increasing the number of relevant data points.  
Let's apply this to our data:

In [ ]:
%%time
# apply clean function with  lambda  function
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 21.3 s, sys: 209 ms, total: 21.5 s
Wall time: 24.8 s


In [ ]:
print(list(data["label"].values))#print values of label

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [ ]:
data.drop(data.loc[data["label"]==2].index,inplace=True)# drop value of 2 in label as it is found least number of times.

In [ ]:

data_clean = data.copy()# make a copy


These steps conclude the cleaning and pre processing. In result, we get this:

In [ ]:
data_clean.head(3)



,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


## Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data_clean["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30434
of       29139
in       23559
and      19744
it       12648
for      12313
on       11231
this     10316
is        9991
with      8493
from      7487
my        7291
that      7020
you       6491
his       6391
at        6352
wa        5467
by        5455
he        5223
after     5124
an        4532
ha        4214
a         4163
year      4125
they      3938
are       3787
be        3460
out       3409
have      3401
one       3285
her       3112
new       2998
but       2968
like      2949
who       2902
up        2847
their     2845
not       2772
what      2737
dtype: int64

In [ ]:
type(word_freq)#check type

pandas.core.series.Series

In [ ]:
print(data_clean["text_clean"].duplicated().sum())

434


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
# Distribution of label
data_clean["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

In [ ]:
data_clean.shape# check shape
data_clean.head(3)# print first 3 rows

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


In [ ]:
data_clean.info()# check types and information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59768 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        59768 non-null  object
 1   label       59768 non-null  int64 
 2   text_clean  59768 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:



y = data_clean['label'] # target

X = data_clean["text_clean"]#feature
print('original shape', X.shape, y.shape)




original shape (59768,) (59768,)


In [ ]:
!pip install scikit_optimize  #install skopt

##Feature creation with TF-IDF
* Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation using  apply TF-IDF using scikit-learn.
* The Pipeline object encapsulates several processing steps into one(we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search or random search functionality in sci-kit learn.

Gridsearch with validation set

In [ ]:
# feature creation and modelling in a single function
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022) # split data
    # Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit

pds = PredefinedSplit(test_fold = split_index)
# apply tranformer ,model using pipline.
pipe = Pipeline([("tfidf", TfidfVectorizer()), ("LogisticRegression", LogisticRegression(verbose=1, solver='liblinear',random_state=42,  penalty='l2',max_iter=1000))])


# define parameter space to test 

params = {
   "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 15),
    'LogisticRegression__C': [0.1, 1],  
    
    
    
}


# here we still use X_train; but the random search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set

pipe_clf = GridSearchCV(
    pipe, params, n_jobs=-1, scoring='roc_auc',cv=pds)
pipe_clf.fit(X, y)


[LibLinear]

GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1,  0])),
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('LogisticRegression',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42,
                                                           solver='liblinear',
                                                           verbose=1))]),
             n_jobs=-1,
             param_grid={'LogisticRegression__C': [0.1, 1],
                         'tfidf__max_df': array([0.3]),
                         'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'tfidf__ngram_range': [(1, 2), (1, 3)]},
             scoring='roc_auc')

In [ ]:
print(pipe_clf.best_params_)

{'LogisticRegression__C': 1, 'tfidf__max_df': 0.3, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


In [ ]:
print('best score {}'.format(pipe_clf.best_score_))

best score 0.8904071306680476


## load  test data and do the same pre processing.

In [ ]:
data__test = pd.read_csv("/content/x_test.csv")# read data
data__test

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
datats_clean = data__test.copy()# make acopy

In [ ]:
data__test.drop('id', axis=1,inplace=True)# drop idt column (it  may not be useful)

In [ ]:
data__test.head(2)# print first 2 columns of dataframe

,text
0,stargazer
1,yeah


In [ ]:
%%time
# Aplly function of clean using   lambda function
data__test["text_clean"] = data__test.loc[data__test["text"].str.len() > 0, "text"]
data__test["text_clean"] = data__test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 15.2 s, sys: 55.3 ms, total: 15.3 s
Wall time: 18.3 s


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
test_pred=pipe_clf.predict_proba(data__test["text_clean"])# prediction on all dataset
test_pred

array([[0.54525894, 0.45474106],
       [0.80909763, 0.19090237],
       [0.35262611, 0.64737389],
       ...,
       [0.83346549, 0.16653451],
       [0.70828652, 0.29171348],
       [0.20797019, 0.79202981]])

Save result in csv file

In [ ]:
submission = pd.DataFrame()

submission['id'] = datats_clean["id"]

submission['label'] = pipe_clf.predict_proba(data__test["text_clean"])[:,1]

submission.to_csv('try78.csv', index=False)

#plan for try 3 Xgb
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for transforming text to numerical using **TfidfVectorizer**,training model(**Xgb**  classifier with  tunning hyperparamter with grid with validation set).
* In this try  I used different cleaning steps on text (**Lemmatization.**),different classifier to increase performmane because **XGBoost** converts weak learners to strong learners,**different tunning hyperparamters**.

* steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation , Lemmatization.



#Observation :
* model with tunning  hyper paramter  with **gridsearch** .
* I expect   that try is the best  because it is strong classifier and has many tuning parameters   
 but  the  private score  of using Lemmatization with **XGB** on kaggle(0.8028 ).
so i will use random forest to increase performance.




In [ ]:
# Read  data

data = pd.read_csv("/content/xy_train.csv", sep=",", na_values=[""])
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
data .info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
data.head(5)# print first 5 rows

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [ ]:
data=data.drop(labels='id', axis=1)# drop idt column (it  may not be useful)

##Cleaning and pre processing

Having consistent and clean data is fundamental for good modeling results. No matter how sophisticated your model the basic principle is: trash in trash out. When dealing with NLP the cleaning and pre processing can differ depending on which model you intend to use. We will use frequency based representation methods for our text. Thus, we usually want to have a pretty thorough manipulation of the input data:

In [ ]:
 
import nltk 
from nltk.stem import 	WordNetLemmatizer# donwnload Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:

nltk.download('punkt')# download punkt from ntlk
nltk.download('stopwords')## download stopwords from ntlk

stemmer = SnowballStemmer("english")# define SnowballStemmer algorithm
stop_words = set(stopwords.words("english"))# define stop_words in english

def _lemmatize( text):# apply lemmatize function.

  toekn_lemma=[]
  for w in text:
      toekn_lemma.append(wordnet_lemmatizer.lemmatize(w))
  return ' '.join(toekn_lemma)
# apply clean function
		
def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and lemmatize
    """
    #applay regular expression(re) is mostly used library for text cleaning
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)
#	Replaces one or many matches with astring using sub RegEx Functions
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
#convert text to lowercase
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    
    text_clean=_lemmatize(words_tokens_lower)
    return text_clean
    
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The `clean_text` function takes a string input and applies a bunch of manipulations to it (described in the code). Check out this example:

This transformation has a few benefits. Removing characters and words that don't hold much meaning reduces the size of our data. Moreover, it can improve prediction performance when modeling by lowering the noise in the data. This is because e.g. stop words like prepositions or punctuation won't allow our model to extract additional information / meaning (at least when using simple models). By stemming and lower casing words we make sure that similar words are treated identically. Thus, we can improve model performance again by increasing the number of relevant data points.  
Let's apply this to our data:

In [ ]:
%%time
# apply clean function with  lambda  function
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 17.8 s, sys: 66.8 ms, total: 17.8 s
Wall time: 18.9 s


In [ ]:
print(list(data["label"].values))#print values of label

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [ ]:
data.drop(data.loc[data["label"]==2].index,inplace=True)# drop value of 2 in label as it is found least number of times.

In [ ]:

data_clean = data.copy()# make a copy


These steps conclude the cleaning and pre processing. In result, we get this:

In [ ]:
data_clean.head(3)



,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


## Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data_clean["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30434
of       29139
in       23559
and      19744
it       12648
for      12313
on       11231
this     10316
is        9991
with      8493
from      7487
my        7291
that      7020
you       6491
his       6391
at        6352
wa        5467
by        5455
he        5223
after     5124
an        4532
ha        4214
a         4163
year      4125
they      3938
are       3787
be        3460
out       3409
have      3401
one       3285
her       3112
new       2998
but       2968
like      2949
who       2902
up        2847
their     2845
not       2772
what      2737
dtype: int64

In [ ]:
type(word_freq)#check type

pandas.core.series.Series

In [ ]:
print(data_clean["text_clean"].duplicated().sum())

434


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
# Distribution of label
data_clean["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

In [ ]:
data_clean.shape# check shape
data_clean.head(3)# print first 3 rows

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


In [ ]:
data_clean.info()# check types and information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59768 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        59768 non-null  object
 1   label       59768 non-null  int64 
 2   text_clean  59768 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:



y = data_clean['label'] # target

X = data_clean["text_clean"]#feature
print('original shape', X.shape, y.shape)




original shape (59768,) (59768,)


In [ ]:
!pip install scikit_optimize  #install skopt

##Feature creation with TF-IDF
* Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation using  apply TF-IDF using scikit-learn.
* The Pipeline object encapsulates several processing steps into one(we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search or random search functionality in sci-kit learn.

Gridsearch with validation set

In [ ]:
# feature creation and modelling in a single function
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022) # split data
    # Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit

pds = PredefinedSplit(test_fold = split_index)
# apply tranformer ,model using pipline.
pipe = Pipeline([("tfidf", TfidfVectorizer(use_idf=True, max_df=0.95,)),("xgb", XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7, 
                    n_estimators=300))])
# define parameter space to test # runtime 35min
params = {
   "tfidf__ngram_range": [(1, 2), (1, 3)],
    #"tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 15),
    'xgb__learning_rate': [0.1, 0.05, 0.01]
   
    
}






# here we still use X_train; but the random search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set

pipe_clf = GridSearchCV(
    pipe, params, n_jobs=-1, scoring='roc_auc',cv=pds)
pipe_clf.fit(X, y)


##load  test data and do the same pre processing.

In [ ]:
data__test = pd.read_csv("/content/x_test.csv")# read data
data__test

In [ ]:
datats_clean = data__test.copy()# make acopy

In [ ]:
data__test.drop('id', axis=1,inplace=True)# drop idt column (it  may not be useful)

In [ ]:
data__test.head(2)# print first 2 columns of dataframe

In [ ]:
%%time
# Aplly function of clean using   lambda function
data__test["text_clean"] = data__test.loc[data__test["text"].str.len() > 0, "text"]
data__test["text_clean"] = data__test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

In [ ]:
print(data__test["text_clean"])# print text after cleaning

In [ ]:
test_pred=pipe_clf.predict_proba(data__test["text_clean"])# prediction on all dataset
test_pred

Save result in csv file

In [ ]:
submission = pd.DataFrame()

submission['id'] = datats_clean["id"]

submission['label'] = pipe_clf.predict_proba(data__test["text_clean"])[:,1]

submission.to_csv('try77.csv', index=False)

#plan for try 4 random forest
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for transforming text to numerical using **TfidfVectorizer**,training model(**Random forest  classifier** with  tunning hyperparamter with random searchwith validation set).

* In this try 
* I used different cleaning steps on text. (**Lemmatization.** ), different classifier to increase performance,different tunning hyperparamters.

* steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation , Lemmatization.



#Observation :
* model with tunning  hyper paramter  with **Randomsearch** .
* I expect   that try is the best one  because Lemmatization doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . 
 and actually  the  private score  of using Lemmatization with **random forest** on kaggle(0.88428 )




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# Read  data
import pandas as pd
data = pd.read_csv("/xy_train.csv", sep=",", na_values=[""])
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
data .info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
data.head(5)# print first 5 rows

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [ ]:
data=data.drop(labels='id', axis=1)# drop idt column (it  may not be useful)

## Cleaning and pre processing

Having consistent and clean data is fundamental for good modeling results. No matter how sophisticated your model the basic principle is: trash in trash out. When dealing with NLP the cleaning and pre processing can differ depending on which model you intend to use. We will use frequency based representation methods for our text. Thus, we usually want to have a pretty thorough manipulation of the input data:

In [ ]:
 
import nltk 
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:

nltk.download('punkt')# download punkt from ntlk
nltk.download('stopwords')## download stopwords from ntlk

stemmer = SnowballStemmer("english")  # define SnowballStemmer algorithm
stop_words = set(stopwords.words("english"))# define stop_words in english

def _lemmatize( text):# apply lemmatize function.

  toekn_lemma=[]
  for w in text:
      toekn_lemma.append(wordnet_lemmatizer.lemmatize(w))
  return ' '.join(toekn_lemma)
# apply clean function
		
def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and lemmatize
    """
    #applay regular expression(re) is mostly used library for text cleaning
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)
#	Replaces one or many matches with astring using sub RegEx Functions
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
#convert text to lowercase
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    
    text_clean=_lemmatize(words_tokens_lower)
    return text_clean
    
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


This transformation has a few benefits. Removing characters and words that don't hold much meaning reduces the size of our data. Moreover, it can improve prediction performance when modeling by lowering the noise in the data. This is because e.g. stop words like prepositions or punctuation won't allow our model to extract additional information / meaning (at least when using simple models). By stemming and lower casing words we make sure that similar words are treated identically. Thus, we can improve model performance again by increasing the number of relevant data points.  
Let's apply this to our data:

In [ ]:
%%time
# apply clean function with  lambda  function
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 19.8 s, sys: 177 ms, total: 20 s
Wall time: 20.2 s


In [ ]:
print(list(data["label"].values))#print values of label

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [ ]:
data.drop(data.loc[data["label"]==2].index,inplace=True)# drop value of 2 in label as it is found least number of times.

In [ ]:

data_clean = data.copy()# make a copy


These steps conclude the cleaning and pre processing. In result, we get this:

In [ ]:
data_clean.head(3)



,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


## Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data_clean["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30434
of       29139
in       23559
and      19744
it       12648
for      12313
on       11231
this     10316
is        9991
with      8493
from      7487
my        7291
that      7020
you       6491
his       6391
at        6352
wa        5467
by        5455
he        5223
after     5124
an        4532
ha        4214
a         4163
year      4125
they      3938
are       3787
be        3460
out       3409
have      3401
one       3285
her       3112
new       2998
but       2968
like      2949
who       2902
up        2847
their     2845
not       2772
what      2737
dtype: int64

In [ ]:
type(word_freq)#check type

pandas.core.series.Series

In [ ]:
print(data_clean["text_clean"].duplicated().sum())

434


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
# Distribution of label
data_clean["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

In [ ]:
data_clean.shape# check shape
data_clean.head(3)# print first 3 rows

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


In [ ]:
data_clean.info()# check types and information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59768 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        59768 non-null  object
 1   label       59768 non-null  int64 
 2   text_clean  59768 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:



y = data_clean['label'] # target

X = data_clean["text_clean"]#feature
print('original shape', X.shape, y.shape)




original shape (59768,) (59768,)


In [ ]:
!pip install scikit_optimize  #install skopt

     |████████████████████████████████| 100 kB 6.9 MB/s 


##Feature creation with TF-IDF
* Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation using  apply TF-IDF using scikit-learn.
* The Pipeline object encapsulates several processing steps into one(we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search or random search functionality in sci-kit learn.

RandomizedSearchCV with validation set

In [ ]:
# feature creation and modelling in a single function
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022) # split data
    # Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit

pds = PredefinedSplit(test_fold = split_index)
# apply tranformer ,model using pipline.
pipe = Pipeline([("tfidf", TfidfVectorizer()),("rand",RandomForestClassifier(n_estimators=500)) ])


# define parameter space to test # runtime 35min
params = {
   "tfidf__ngram_range": [(1, 2)],
    #"tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5),
    
}


# here we still use X_train; but the random search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set

pipe_clf = RandomizedSearchCV(
    pipe, params, n_jobs=-1, scoring='roc_auc',cv=pds)
pipe_clf.fit(X, y)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1,  0])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('rand',
                                              RandomForestClassifier(n_estimators=500))]),
                   n_jobs=-1,
                   param_distributions={'tfidf__min_df': array([0, 1, 2, 3, 4]),
                                        'tfidf__ngram_range': [(1, 2)]},
                   scoring='roc_auc')

In [ ]:
print(pipe_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 2}


In [ ]:
print('best score {}'.format(pipe_clf.best_score_))

best score 0.8723321835298642


In [ ]:
data__test = pd.read_csv("/x_test.csv")# read data
data__test

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
datats_clean = data__test.copy()# make acopy

In [ ]:
data__test.drop('id', axis=1,inplace=True)# drop idt column (it  may not be useful)

In [ ]:
data__test.head(2)# print first 2 columns of dataframe

,text
0,stargazer
1,yeah


In [ ]:
%%time
# Aplly function of clean using   lambda function
data__test["text_clean"] = data__test.loc[data__test["text"].str.len() > 0, "text"]
data__test["text_clean"] = data__test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 10.5 s, sys: 44.1 ms, total: 10.6 s
Wall time: 10.6 s


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
test_pred=pipe_clf.predict_proba(data__test["text_clean"])# prediction on all dataset
test_pred

array([[0.91517475, 0.08482525],
       [0.91453082, 0.08546918],
       [0.36726136, 0.63273864],
       ...,
       [0.816     , 0.184     ],
       [0.6618    , 0.3382    ],
       [0.34592457, 0.65407543]])

Save result in csv file

In [ ]:
submission = pd.DataFrame()

submission['id'] = datats_clean["id"]

submission['label'] = pipe_clf.predict_proba(data__test["text_clean"])[:,1]

submission.to_csv('try70.csv', index=False)

# plan for try 5 logistic
* I used Pipeline as it is useful ,there are often a fixed sequence of steps in processing the data.
* It is used to chain multiple estimators into one and hence, automate the machine learning process.
* the steps in  pipline in our case(I used it for transforming text to numerical using countVectorizer,training model(logistic regression  classifier with  tunning hyperparamter **with random with validation set**).

* In this try 
* I used different cleaning  steps (Lemmatization)on text.
different **range on n-gram**,**max dif,min dif**, on **character level**.different vectorizer(**count vectorizer**),**different hyperparamters.** 

* steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation , Lemmatization.



#Observation :
* model with tunning  hyper paramter  **with randomsearch** .
* I expect   that try is little better than one using  vectorizer(Tf-idf) because  Tf-idf is higher performance than **count vectorizer**. 
 and actually  the  private score  of using Lemmatization with logistic on kaggle(0.80628 )




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path


In [ ]:
# Read  data

data = pd.read_csv("/xy_train.csv", sep=",", na_values=[""])
data

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0
...,...,...,...
59995,70046,Finish Sniper Simo H盲yh盲 during the invasion o...,0
59996,189377,Nigerian Prince Scam took $110K from Kansas ma...,1
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy...,0
59998,140950,Julius Caesar upon realizing that everyone in ...,0


In [ ]:
data .info()# checking types:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60000 non-null  int64 
 1   text    60000 non-null  object
 2   label   60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [ ]:
data.head(5)# print first 5 rows

,id,text,label
0,265723,A group of friends began to volunteer at a hom...,0
1,284269,British Prime Minister @Theresa_May on Nerve A...,0
2,207715,"In 1961, Goodyear released a kit that allows P...",0
3,551106,"Happy Birthday, Bob Barker! The Price Is Right...",0
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Y...",0


In [ ]:
data=data.drop(labels='id', axis=1)# drop idt column (it  may not be useful)

## Cleaning and pre processing

Having consistent and clean data is fundamental for good modeling results. No matter how sophisticated your model the basic principle is: trash in trash out. When dealing with NLP the cleaning and pre processing can differ depending on which model you intend to use. We will use frequency based representation methods for our text. Thus, we usually want to have a pretty thorough manipulation of the input data:

In [ ]:
 
import nltk 
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
 
nltk.download('punkt')# download punkt from ntlk
nltk.download('stopwords')## download stopwords from ntlk

stemmer = SnowballStemmer("english")# define SnowballStemmer algorithm
stop_words = set(stopwords.words("english"))# define stop_words in english

def _lemmatize( text):# apply lemmatize function.

  toekn_lemma=[]
  for w in text:
      toekn_lemma.append(wordnet_lemmatizer.lemmatize(w))
  return ' '.join(toekn_lemma)
# apply clean function
		
def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and lemmatize
    """
    #applay regular expression(re) is mostly used library for text cleaning
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)
#	Replaces one or many matches with astring using sub RegEx Functions
    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)
#convert text to lowercase
    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    
    text_clean=_lemmatize(words_tokens_lower)
    return text_clean
    
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


This transformation has a few benefits. Removing characters and words that don't hold much meaning reduces the size of our data. Moreover, it can improve prediction performance when modeling by lowering the noise in the data. This is because e.g. stop words like prepositions or punctuation won't allow our model to extract additional information / meaning (at least when using simple models). By stemming and lower casing words we make sure that similar words are treated identically. Thus, we can improve model performance again by increasing the number of relevant data points.  
Let's apply this to our data:

In [ ]:
%%time
# apply clean function with  lambda  function
data["text_clean"] = data.loc[data["text"].str.len() > 0, "text"]
data["text_clean"] = data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 18.1 s, sys: 99.3 ms, total: 18.2 s
Wall time: 18.3 s


In [ ]:
print(list(data["label"].values))#print values of label

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 

In [ ]:
data.drop(data.loc[data["label"]==2].index,inplace=True)# drop value of 2 in label as it is found least number of times.

In [ ]:

data_clean = data.copy()# make a copy


These steps conclude the cleaning and pre processing. In result, we get this:

In [ ]:
data_clean.head(3)



,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


## Descriptive analysis

Even though we deal with texts, we should still use some descriptive analysis to get a better understanding of the data:

In [ ]:
from bokeh.models import NumeralTickFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
# Word Frequency of most common words
word_freq = pd.Series(" ".join(data_clean["text_clean"]).split()).value_counts()
word_freq[1:40]

to       30434
of       29139
in       23559
and      19744
it       12648
for      12313
on       11231
this     10316
is        9991
with      8493
from      7487
my        7291
that      7020
you       6491
his       6391
at        6352
wa        5467
by        5455
he        5223
after     5124
an        4532
ha        4214
a         4163
year      4125
they      3938
are       3787
be        3460
out       3409
have      3401
one       3285
her       3112
new       2998
but       2968
like      2949
who       2902
up        2847
their     2845
not       2772
what      2737
dtype: int64

In [ ]:
type(word_freq)#check type

pandas.core.series.Series

In [ ]:
print(data_clean["text_clean"].duplicated().sum())

434


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
# Distribution of label
data_clean["label"].value_counts(normalize=True)

0    0.538281
1    0.461719
Name: label, dtype: float64

In [ ]:
data_clean.shape# check shape
data_clean.head(3)# print first 3 rows

,text,label,text_clean
0,A group of friends began to volunteer at a hom...,0,group of friend began to volunteer at homeless...
1,British Prime Minister @Theresa_May on Nerve A...,0,british prime minister theresa may on nerve at...
2,"In 1961, Goodyear released a kit that allows P...",0,in goodyear released kit that allows p to be b...


In [ ]:
data_clean.info()# check types and information

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59768 entries, 0 to 59999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        59768 non-null  object
 1   label       59768 non-null  int64 
 2   text_clean  59768 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8+ MB


In [ ]:



y = data_clean['label'] # target

X = data_clean["text_clean"]#feature
print('original shape', X.shape, y.shape)




original shape (59768,) (59768,)


In [ ]:
!pip install scikit_optimize  #install skopt

##Feature creation with TF-IDF
* Because classification models cannot deal with text data directly, we need to convert our comments to a numeric representation using  apply TF-IDF using scikit-learn.
* The Pipeline object encapsulates several processing steps into one(we will compare different parameters for feature creation as well as modeling. We can achieve this by making use of the pipeline and grid search or random search functionality in sci-kit learn.

RandomizedSearchCV  with validation set

In [ ]:
# feature creation and modelling in a single function
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y, train_size = 0.8, stratify = y, random_state = 2022) # split data
    # Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X.index]

# Use the list to create PredefinedSplit

pds = PredefinedSplit(test_fold = split_index)
# apply tranformer ,model using pipline.
pipe = Pipeline([("count", CountVectorizer(analyzer='char')), ("LogisticRegression", LogisticRegression(verbose=1, solver='liblinear',random_state=42,  penalty='l2',max_iter=1000))])


# define parameter space to test 

params = {
   "count__ngram_range": [(1,2), (1, 3)],
    "count__max_df": np.arange(0.3, 0.5),
    "count__min_df": np.arange(5, 9),
    'LogisticRegression__C': [0.01,0.1],  
    
    
    
}


# here we still use X_train; but the random search model
# will use our predefined split internally to determine 
# which sample belongs to the validation set

pipe_clf = RandomizedSearchCV(
    pipe, params, n_jobs=-1, scoring='roc_auc',cv=pds)
pipe_clf.fit(X, y)


[LibLinear]

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1,  0])),
                   estimator=Pipeline(steps=[('count',
                                              CountVectorizer(analyzer='char')),
                                             ('LogisticRegression',
                                              LogisticRegression(max_iter=1000,
                                                                 random_state=42,
                                                                 solver='liblinear',
                                                                 verbose=1))]),
                   n_jobs=-1,
                   param_distributions={'LogisticRegression__C': [0.01, 0.1],
                                        'count__max_df': array([0.3]),
                                        'count__min_df': array([5, 6, 7, 8]),
                                        'count__ngram_range': [(1, 2), (1, 3)]},
                   scoring='roc_auc')

In [ ]:
print(pipe_clf.best_params_)

{'count__ngram_range': (1, 3), 'count__min_df': 8, 'count__max_df': 0.3, 'LogisticRegression__C': 0.1}


In [ ]:
print('best score {}'.format(pipe_clf.best_score_))

best score 0.8503821438773422


## load  test data and do the same pre processing.

In [ ]:
data__test = pd.read_csv("/x_test.csv")# read data
data__test

,id,text
0,0,stargazer
1,1,yeah
2,2,PD: Phoenix car thief gets instructions from Y...
3,3,"As Trump Accuses Iran, He Has One Problem: His..."
4,4,"""Believers"" - Hezbollah 2011"
...,...,...
59146,59146,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House...
59148,59148,Napoleon returns from his exile on the island ...
59149,59149,Deep down he always wanted to be a ballet dancer


In [ ]:
datats_clean = data__test.copy()# make acopy

In [ ]:
data__test.drop('id', axis=1,inplace=True)# drop idt column (it  may not be useful)

In [ ]:
data__test.head(2)# print first 2 columns of dataframe

,text
0,stargazer
1,yeah


In [ ]:
%%time
# Aplly function of clean using   lambda function
data__test["text_clean"] = data__test.loc[data__test["text"].str.len() > 0, "text"]
data__test["text_clean"] = data__test["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x)


CPU times: user 9.95 s, sys: 52 ms, total: 10 s
Wall time: 10 s


In [ ]:
# list most uncommon words
word_freq[-10:].reset_index(name="freq")

,index,freq
0,irhoud,1
1,kya,1
2,multidisciplinary,1
3,shippikh,1
4,bohush,1
5,ruthie,1
6,byelorussian,1
7,transcaucasian,1
8,kilcullen,1
9,wahre,1


In [ ]:
test_pred=pipe_clf.predict_proba(data__test["text_clean"])# prediction on all dataset
test_pred

array([[0.36159147, 0.63840853],
       [0.38404759, 0.61595241],
       [0.30378063, 0.69621937],
       ...,
       [0.70407185, 0.29592815],
       [0.66336114, 0.33663886],
       [0.11608782, 0.88391218]])

Save result in csv file

In [ ]:
submission = pd.DataFrame()

submission['id'] = datats_clean["id"]

submission['label'] = pipe_clf.predict_proba(data__test["text_clean"])[:,1]

submission.to_csv('try73.csv', index=False)

## Questions

* 1-What is the difference between Character n-gram and Word n-gram? Which one tends to suffer more from the OOV issue?

* Character N-Grams it is split each words based on 
* character  word N-Grams it is split whole text based on words

* For a given document in a language, there are more character N-grams repetitions than word n-gram repetitions

* character N-grams split highlights some common properties that a word n-gram split does not

* character N-grams make languages more comparable revealing more repeated.
Word n-gram tends to suffer more from OOV (Out-Of-Vocabulary) issue because of the new words that presented in the testing dataset and not appear in training dataset


* 2-What is the difference between stop word removal and stemming? Are these techniques language-dependent?



* Stop word removal and stemming are commonly used method in indexing the different between them are

* Stop word removal Some common words that are present in text but do not contribute in the meaning of a sentence. Such words are not at all important for the purpose of information retrieval or natural language processing. The most common stopwords are ‘the’ and ‘a’.

* Stemming is a technique used to extract the base form of the words by removing affixes from them. It is just like cutting down the branches of a tree to its stems. For example, the stem of the words eating, eats, eaten is eat.
yes, these techniques are language-dependent




* 3-Is tokenization techniques language dependent? Why?

Tokenization is breaking the raw text into small chunks and helps in interpreting the meaning of the text by analyzing the sequence of the words so it is language dependent
becouse it needs to know meaning of each word to split it depend on language for example: wasn't>> wasnt>>was n't>>wasn t >> these all possable of tokenization but some of these doesn't have meaning in the language

* 4-What is the difference between count vectorizer and tf-idf vectorizer? Would it be feasible to use all possible n-grams? If not, how should you select them?

TfidfVectorizer and CountVectorizer both are methods for converting text data into vectors as model  is able to deal with numeric data.

In CountVectorizer we only count the number of times a word appears in the document which results are interested in of most frequent words. 

* In TfidfVectorizer we consider overall document weightage of a word. It helps us in dealing with most frequent words.  TfidfVectorizer weights the word counts by a measure of how often they appear in the documents.

* it isn't feasible to use all possible n-grams
N-gram range sets if features to be used to characterize texts will be:Unigrams or words (n-gram size = 1)Bigrams or terms compounded by two words (n-gram size = 2)Trigrams or terms compounded by up to three words (n-gram size = 3)
* I should select between them depend on problem like :Sentiment Analysis, setting n-gram ranges that use bigrams or trigrams can greatly improve the accuracy of classification,as they can capture more complex expressions formed by the composition of morethan one word. The rationale is that in Sentiment Analysis the outcome depends not only on the frequency of words but also on how they are combined:good has a different meaning alone than when preceded by a not as in not good.